# Wizard Crawler 
*(noun)*
1. Crawls threads in search of comments from the wizard, /u/XiuathoTheWizard, and compiles them into a .json file.
2. Converts the result into an easy to navigate html.
3. For more info see the [Github repository](https://github.com/XkF21WNJ/WizardTree)

Credits to /u/XiuathoTheWizard and everyone else who contributed to the story.

Code by /u/XkF21WNJ.

### Crawl Thread

In [1]:
#Imports
import praw

#Definitions
def IsWizard(comment):
    return str(comment.author) == "XiuathoTheWizard"

def YieldAll(comments):
    for c in comments:
        if isinstance(c, praw.objects.MoreComments):
            for x in YieldAll(c.comments()): yield x
        else: yield c

def FindWizard(comments):
    for c in YieldAll(comments):
        if IsWizard(c): yield c

def BuildTree(comments):
    for c in FindWizard(comments):
        return {"description": c.body, "actions" : list(GetActions(c)), "id" : c.id, "url" : c.permalink}

def GetActions(comment):
    for r in YieldAll(comment.replies):
        tree = BuildTree(r.replies)
        if tree == None: continue;
            
        yield {"description": r.body, "actor" : str(r.author), "consequence" : tree, "id" : r.id, "url" : r.permalink}

#Initialisation
r = praw.Reddit(user_agent='WizardCrawler/0.1 (by /u/XkF21WNJ)')
        
#Crawl submission
submission = r.get_submission(submission_id='3xyesc')    

#Parse thread
tree = BuildTree(submission.comments)

### Import / Export

#### Import tree from file

In [2]:
import json

f = open("Output/WizardTree.json",'r')
tree = json.load(f)
f.close()

#### Export tree to file

In [7]:
import json

f = open("Output/WizardTree.json",'w')
json.dump(tree, f, indent=4)
f.close()

### HTML conversion

In [40]:
#Imports
from html import HTML
import markdown as md

#Definitions
def Truncate(text):
    return text[:60] + (text[60:] and '..')

def ParseMarkDown(text, el):
    text=md.markdown(text)
    el.text(text, escape=False)

def ParseEvent(event, index, previous=None):
    div = body.div(klass = 'event')
    div.a('', name = event['id'])
    
    links = div.div(klass = 'link-container')
    if previous != None:     
        links.a('Go to Previous', 
                href = '#'+previous['id'],
                klass = 'left-link')
    else:
        links.a('Go to Index', href = '#index', klass = 'left-link')
    links.a('Go to Original', href = event['url'], klass = 'right-link')
        
    ParseMarkDown(event['description'], div)
    
    for action in event['actions']:
        ParseAction(action, div.ul(), event, index)

def ParseAction(action, ul, parent, index):
    href = '#'+action['consequence']['id']
    
    indi = index.li()
    indi.a(Truncate(action['description']), href = href, klass='action')
    users.append(action['actor'])
    
    li = ul.li(klass = 'action')
    a = li.a(href = href,
             style = "display: block;", 
             title = '/u/'+ action['actor'])
    
    ParseMarkDown(action['description'], a.span())
    ParseEvent(action['consequence'], index, parent)
    
#Styling
css = """
    body {
        font-family: arial, sans-serif;
        width: 40em;
        margin-left:auto;
        margin-right:auto;
    }
    
    p {
        text-align: justiy;
        -moz-hyphens: auto;
        hyphens: auto;
    }
    
    .event {
        margin-bottom: 1000ex;
    }
    
    .event > p {
        font-family: Georgia, serif;
        font-size: larger;
        line-height: 1.5em;
    }
    
    .action p {
        margin-top: 0.25em;
        margin-bottom: 0.25em;
    }
    
    .action {
        font-family: consolas, monospace;
        margin: 0 0 2ex 0;
    }
    
    .link-container {
        font-family: consolas, monospace;
        display: flex;
    }
    
    .left-link{
        width: 40ex;
    }
    
    .right-link{
        flex-grow: 1;
        text-align: right;
    }
"""

#Build HTML file
doc = HTML()
doc.text("<!DOCTYPE html>", escape=False)

html = doc.html()
header = html.head()
header.meta(charset = "UTF-8")
header.title("XiuathoTheWizard's Epic Quest.")
header.style(css, escape = False)

body = html.body()
index = HTML().ul()
users = list()

#Parse tree
ParseEvent(tree, index)

#Add index
body.a('', name = 'index')
body.h1('Index')
body.div(index, escape = False)

#Add credits
body.a('', name = 'credits')
body.h1('Credits')
body.p("A special thanks to /u/XiuathoTheWizard for creating the story, and bringing the /r/TalesFromTechSupport a tale that won't soon be forgotten.")
body.p("Also thanks to all the commenters appearing in the story:")
uniq_users = list(set(users))
uniq_users.sort(key = users.count, reverse = True)
users_ul = body.ul()
for user in uniq_users: users_ul.li("/u/"+user)

body.p("Powered by python, coded by /u/XkF21WNJ.")

#Export to file
f = open("Output/WizardTree.html", 'w')
f.write(unicode(doc).encode('utf-8'))
f.close()